Source: http://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#sphx-glr-intermediate-seq2seq-translation-tutorial-py

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
%matplotlib inline

use_cuda = torch.cuda.is_available()

In [2]:
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    #put a space between punctuation, so not included in word
    s = re.sub(r"([.!?])", r" \1", s)
    #remove things that are not letters or punctuation
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def unicodeToAscii(s):
    # basically removes non-english standard alphabet letters and replaces with equivalent.
    # so no accented letters => less to learn
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def readLangs(input_file_location, lang1_name="English", lang2_name="French"):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(input_file_location, encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    input_lang = Lang(lang1_name)
    output_lang = Lang(lang2_name)

    return input_lang, output_lang, pairs

In [3]:
# To make learning easier, we will limit to sentences less than max length 
# in either language
# And only take sentences that start with certain prefixes
# Removed punctuation b/c filtered out in our normalize function

MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[0].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


def prepareData(input_file_location, lang1_name="English", 
                lang2_name="French"):
    input_lang, output_lang, pairs = readLangs(input_file_location, lang1_name,
                                              lang2_name)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [4]:
#start of sentence
SOS_token = 0
# end of sentence
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.word2count = {}
        self.n_words = 2
        
    def addSentence(self, sentence):
        for word in sentence.split(" "):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
input_lang, output_lang, pairs = prepareData("../../../data/text/fra-eng/fra.txt")
print(random.choice(pairs))

Reading lines...
Read 149861 sentence pairs
Trimmed to 11589 sentence pairs
Counting words...
Counted words:
English 3016
French 4634
['he is clumsy with his hands .', 'il est maladroit de ses mains .']


In [6]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(" ")]

def variableFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    # make it 1 column with number of rows equal to words in sentence
    result = Variable(torch.LongTensor(indexes).view(-1, 1))
    if use_cuda:
        return result.cuda()
    else:
        return result

def variableFromPair(pair):
    input_variable = variableFromSentence(input_lang, pair[0])
    output_variable = variableFromSentence(output_lang, pair[1])
    return (input_variable, output_variable)

In [7]:
class EncoderRNN(nn.Module):
    """
    Simple encoder network that embeds the character and then feeds through a GRU
    """
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden
    
    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result
        
class AttnDecoderRNN(nn.Module):
    """
    Attn Decoder
    1. Need max length because learning which input words to attend to
    And thus need to know the maximum number of words could attend to
    2. The attn_weights tell us how much to weight each input word - in this case French,
       In order to predict the english word.
    """
    def __init__(self, input_size, hidden_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        self.relu = nn.ReLU()
        self.gru = nn.GRU(hidden_size, hidden_size)
        # note input and output same size
        self.linear = nn.Linear(hidden_size, input_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.attn_layer = nn.Linear(2 * self.hidden_size, MAX_LENGTH)
        self.out_layer = nn.Linear(self.hidden_size, self.input_size)
        self.attn_combined_layer = nn.Linear(2 * self.hidden_size, self.hidden_size)
    
    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        attn = self.attn_layer(torch.cat((embedded[0], hidden[0]),dim=1))
        attn_weights = self.softmax(attn)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), 
                                encoder_outputs.unsqueeze(0))
        attn_combined = torch.cat((embedded[0], attn_applied[0]), 1)
        attn_combined = self.relu(self.attn_combined_layer(attn_combined))
        output, hidden = self.gru(attn_combined, hidden)
        output = self.softmax(self.out_layer(output[0]))
        return output, hidden, attn_weights
    
    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result        

In [8]:
teacher_forcing_ratio = 0.5

def train(input_variable, target_variable, encoder, decoder, encoder_optimizer,
         decoder_optimizer, criterion, max_length=MAX_LENGTH):
    
    encoder_hidden = encoder.initHidden()
    loss = 0
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]
    
    # Here we are feeding in the english words to get the final hidden state 
    # for the decoder
    for i in range(input_length):
        encoder_ouput, encoder_hidden = encoder.forward(input_variable[i], encoder_hidden)
        
    decoder_hidden = encoder_hidden
    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    # Here we take the final hidden state from the encoder
    # And feed it to decoder
    # We also give decoder the word to predict the next word starting with SOS token
    # If use teacher forcing then give it the truth, otherwise give it prediction
    if use_teacher_forcing:
        for i in range(target_length):
            decoder_output, decoder_hidden = decoder.forward(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_variable[i])
            decoder_input = target_variable[i]
            
    else:
        for i in range(target_length):
            decoder_output, decoder_hidden = decoder.forward(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_variable[i])
            topv, topi = decoder_output.data.topk(1)
            top_predicted = topi[0][0]
            
            decoder_input = Variable(torch.LongTensor([[top_predicted]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            if top_predicted == EOS_token:
                break
                
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0] / target_length

In [9]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [10]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [11]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100,
               learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    #get our training data
    training_pairs = [variableFromPair(random.choice(pairs))
                     for i in range(n_iters)]
    
    criterion = nn.NLLLoss()
    
    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter-1]
        input_variable = training_pair[0]
        target_variable = training_pair[1]
        
        loss = train(input_variable, target_variable, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            
    showPlot(plot_losses)

In [12]:
hidden_size = 256
encoder = EncoderRNN(input_lang.n_words, hidden_size)
decoder = DecoderRNN(output_lang.n_words, hidden_size)

if use_cuda:
    encoder = encoder.cuda()
    decoder = decoder.cuda()
    
trainIters(encoder, decoder, 75000, print_every=5000)

2m 16s (- 31m 45s) (5000 6%) 3.5147
4m 31s (- 29m 25s) (10000 13%) 2.9476
6m 49s (- 27m 16s) (15000 20%) 2.6149
9m 7s (- 25m 4s) (20000 26%) 2.3670
11m 26s (- 22m 52s) (25000 33%) 2.1233
13m 45s (- 20m 37s) (30000 40%) 1.9208
16m 4s (- 18m 22s) (35000 46%) 1.7785
18m 23s (- 16m 5s) (40000 53%) 1.6309
20m 44s (- 13m 49s) (45000 60%) 1.4921
23m 3s (- 11m 31s) (50000 66%) 1.3803
25m 23s (- 9m 13s) (55000 73%) 1.2808
27m 43s (- 6m 55s) (60000 80%) 1.1962
30m 3s (- 4m 37s) (65000 86%) 1.0892
32m 23s (- 2m 18s) (70000 93%) 1.0039
34m 43s (- 0m 0s) (75000 100%) 0.9792


In [30]:
torch.save(encoder.state_dict(), "../models/encoder.state")
torch.save(decoder.state_dict(), "../models/decoder.state")

In [22]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    
    encoder_hidden = encoder.initHidden()
    input_variable = variableFromSentence(input_lang, sentence)
    input_length = input_variable.size()[0]

    for i in range(input_length):
        encoder_ouput, encoder_hidden = encoder.forward(input_variable[i], encoder_hidden)
        
    decoder_hidden = encoder_hidden
    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    decoded_words = []
    for i in range(MAX_LENGTH):
        decoder_output, decoder_hidden = decoder.forward(decoder_input, decoder_hidden)
        topv, topi = decoder_output.data.topk(1)
        top_predicted = topi[0][0]

        decoder_input = Variable(torch.LongTensor([[top_predicted]]))
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input
        if top_predicted == EOS_token:
            decoded_words.append("<EOS>")
            break
        else:
            decoded_words.append(output_lang.index2word[top_predicted])
    return decoded_words

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print(">", pair[0])
        print("=", pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        print("<", " ".join(output_words))
        print('')

In [23]:
evaluateRandomly(encoder, decoder)

> i m worried .
= je me fais du souci .
< je fais du souci . <EOS>

> i m really flattered to hear that .
= je suis tres flatte d entendre cela .
< je suis vraiment d d entendre cela . <EOS>

> i m very much in favor of this .
= je suis tres en faveur de cela .
< je suis tres en en faveur de . . <EOS>

> i m going to miss you .
= vous allez me manquer .
< tu vas me manquer . <EOS>

> you re not very funny .
= tu n es pas tres amusant .
< vous n etes pas tres amusante . <EOS>

> i m not giving up yet .
= je ne vais pas encore abandonner .
< je ne gobe pas encore ca . <EOS>

> i m not mad .
= je ne suis pas fou .
< je ne suis pas fou . <EOS>

> they re not all busy .
= elles ne sont pas toutes occupees .
< ils ne sont pas tous occupes . <EOS>

> you re not a loser .
= vous n etes pas un perdant .
< vous n etes pas un perdant . <EOS>

> i m going to hit the hay .
= je vais me pieuter .
< je vais me pieuter . <EOS>



In [28]:
evaluate(encoder, decoder, "i m going to teach .")

['je', 'vais', 'me', 'battre', '.', '<EOS>']